In [1]:
from selenium import webdriver
from urllib.parse import urlparse
import html
from time import sleep
from multiprocessing import Pool, Lock, Value
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import time
import threading
from selenium.common.exceptions import NoSuchElementException

In [2]:
def books_id_ath(url_current, driver, DELAY = 0.5):
    books_id = set()
    driver.get(url_current)
    prh = driver.execute_script("return document.body.scrollHeight")
    nth = prh
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(DELAY)
        prh = nth
        nth = driver.execute_script("return document.body.scrollHeight")
        if prh == nth:
            try:
                driver.execute_script("window.scrollTo(0, -40);")
                next_page = driver.find_element_by_class_name('ty').click()
                sleep(DELAY)
            except NoSuchElementException:
                break
    nwid = [i.get_attribute('data-book') for i in driver.find_elements_by_class_name('gf')]
    books_id.update(set(nwid))
    return books_id

In [3]:
author_id = {
  "Дарья Донцова":  29369,
  "Джеймс Роллинс": 29442,
  "Макс Фрай":      102994,
  "Эрин Хантер":    26149,
  "Дмитрий Емец":   35952
}

url_format = "https://www.bookvoed.ru/author/books?id={author_id}"
url_params = dict()
ids_athrs = dict()

with webdriver.Chrome() as driver:
    for author, _id in author_id.items():
        url_params['author_id'] = _id
        url_current = url_format.format(**url_params)
        ids_athrs[author] = books_id_ath(url_current, driver)

In [19]:
url_format = "https://www.bookvoed.ru/book?id={book_id}"
AGE_MAP = { 'ov': '0+', 'pv': '6+', 'qv': '12+', 'rv': '16+', 'sv': '18+'}

def extract_book_info(id_book):
    url_params['book_id'] =id_book
    url_current = url_format.format(**url_params)
    book_html = requests.get(url_current).text
    soup = BeautifulSoup(book_html, 'html.parser')
    book_info = {
        "ID": id_book,
        "Обложка": soup.find('img', class_ = 'tf').attrs['src'],
        "Рейтинг": float(soup.find('div', class_ = 'af').attrs['style'].split(' ')[1][:-1]),
    }
    #
    t = soup.find('div', class_ = "He xe ")
    if t is not None:
        tt = t.find('a', class_ = 'Ke Me ')
        if tt is not None:
            ttt = tt.text[1:-1]
            if ttt != '':
                book_info["Понравилось"] = int(ttt)
        tt = t.find('a', class_ = 'Ke Le ff')
        if tt is not None:
            ttt = tt.text[1:-1]
            if ttt != '':
                book_info["В закладки"] = int(ttt)
        tt = t.find('a', class_ = 'Ke Oe ')
        if tt is not None:
            ttt = tt.text[1:-1]
            if ttt != '':
                book_info["Не понравилось"] = int(ttt)
    #
    #Description
    t = soup.find('div', class_ = 'lw')
    if t is not None:
        book_info["Описание"] = t.text[:soup.find('div', class_ = 'lw').text.rfind('далее')][1:]
    #
    #Name
    t = soup.find('div', class_ = 'lw')
    if t is not None:
        book_info['Название'] = t.text[:len(soup.find('h1', itemprop = 'name').find('span').text)]
    #
    #age
    age = soup.find('div', class_='l aw').find('div', class_='nM')['class'][0]
    book_info['Возраст'] = AGE_MAP[age]
    #
    #price
    t = soup.find('div', class_ = 'Hu Wu')
    if t is not None:
        book_info["Цена"] = float(t.text.split(' ')[0][1:])
    #
    #table
    table = soup.find('table', class_='tw')
    rows = table.find_all('tr')
    data = [list(map(lambda x: x.text, row.find_all('td'))) for row in rows]
    for i in data:
        book_info[i[0][:-1]] = i[1]
    #
    return book_info
    

In [20]:
extract_book_info(9340993)

{'ID': 9340993,
 'ISBN': '978-5-04-100203-9',
 'Автор': 'Донцова Дарья Аркадьевна',
 'В базе': 'Э.ИД(м).Шоколадное пугало',
 'В закладки': 30,
 'Возраст': '16+',
 'Год': '2019',
 'Издательство': 'Эксмо',
 'Код': '1524330',
 'Название': '\nЕсли вы заботитесь о св',
 'Обложка': '/files/1836/59/23/24/6.jpeg',
 'Описание': 'Если вы заботитесь о своем здоровье, не читайте до обеда книг по оккультизму и магии! Эх, да только некому было дать такой совет Владимиру Сиракузову, когда он начал увлекаться эзотерической литературой. Вот и снесло преуспевающему бизнесмену крышу – сам дьявол собственной персоной начал к нему являться. Татьяна Сергеева, начальница особой бригады, к которой он обратился за помощью, так и решила – напрочь снесло. \nНо что это достает Сиракузов из портфеля? Что это за лист пергамента с печатью и бурым пятном! Да, подтверждает Владимир, это договор с дьяволом о продаже души. Скрепленный кровью. Экспертиза показала – его, Сиракузова, кровью. Вот это поворот! Но Танюше по п

In [8]:
all_ids = set()
for i,j in ids_athrs.items():
    all_ids.update(j)
print(len(all_ids))

1774


In [21]:
%%time

n_processed = 0
result = []
lock = threading.Lock()

def func_wrapper(uid):
    res = extract_book_info(uid) 
    global n_processed
    global result
    with lock:
        n_processed += 1
        if n_processed % 10 == 0:
            print(f"\r{n_processed} objects are processed...", end='', flush=True)
        result.append(res)
    return res

workerthreadlist=[]

for i in all_ids:
    my_thread = threading.Thread(target=func_wrapper, args=(i,))
    workerthreadlist.append(my_thread)
    my_thread.start()

for x in range(0,len(all_ids)):
    workerthreadlist[x].join()
    
df = pd.DataFrame(result)
df.sort_values(by=['ID'], inplace=True)

with open('hw_3.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)

1770 objects are processed...Wall time: 6min 12s
